In [1]:
from utils.models import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

import math

import os

import pandas as pd

import numpy as np

## Load

In [2]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDataset(
        "train_data/train.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=8,
    shuffle=True,
)
val_dataloader = DataLoader(
    RatingsDataset(
        "train_data/validation.csv",
        "user_id",
        "movie_id",
        "rating",
    ),
    batch_size=1024,
    num_workers=4,
    shuffle=True,
)

In [3]:
import torch
import torch.nn as nn
import torch.optim as opt

In [4]:
def define_model(k_gmf, k_mlp, layer_sizes, alpha=0.5):
    return NeuFM(
        GMF(users, movies, k_gmf),
        MLP(users, movies, k_mlp, layer_sizes),
        alpha=alpha,
    ).cuda()

In [5]:
def train(model, lr, epochs, train_dataloader, val_dataloader=None, show_loss=True):
    criterion = nn.MSELoss().cuda()
    optimizer = opt.Adam(model.parameters(), lr=lr)
    
    avg_losses = []
    
    model_state = None
    best_epoch = 0
    prev_val_loss = math.inf
    for epoch in tqdm(range(epochs)):
        n_batches = len(train_dataloader)
        avg_loss = 0
        val_loss = 0

        # Train step
        for i_batch, (vus, vis, rs) in enumerate(train_dataloader):
            vus = vus.cuda()
            vis = vis.cuda()
            rs = rs.cuda()

            optimizer.zero_grad()
            y_hat = model(vus, vis)

            loss = criterion(y_hat, rs)
            loss.backward()
            optimizer.step()

            avg_loss += math.sqrt(float(loss.detach().cpu()))
        
        avg_loss /= n_batches
        
        # Val step
        if val_dataloader is not None:
            with torch.no_grad():
                for val_vus, val_vis, val_rs in val_dataloader:
                    val_vus = val_vus.cuda()
                    val_vis = val_vis.cuda()
                    val_rs = val_rs.cuda()

                    val_pred = model(val_vus, val_vis)
                    val_loss += math.sqrt(float(criterion(val_pred, val_rs).detach().cpu()))

            val_loss /= len(val_dataloader)
            if show_loss:
                print(f"epoch: {epoch+1}, train_loss: {avg_loss}, val_loss: {val_loss}")
            
            avg_losses.append([avg_loss, val_loss])
        else:
            val_loss = avg_loss
            if show_loss:
                print(f"epoch: {epoch+1}, train_loss: {avg_loss}")
            
            avg_losses.append([avg_loss])
                  
        if val_loss < prev_val_loss:
            prev_val_loss = val_loss
            model_state = model.state_dict()
            best_epoch = epoch
        
    return best_epoch, prev_val_loss, model_state, avg_losses

In [6]:
def fit(
    train_dataloader, val_dataloader,
    k_gmf, k_mlp, layer_sizes, alpha=0.5, lr=0.0005, epochs=40, 
    weight_path="/home/", run_number=1, random_state=None, show_loss=True
):
    if random_state is not None:
        torch.manual_seed(random_state)
        
    model = define_model(k_gmf, k_mlp, layer_sizes, alpha)
    
    best_epoch, val_loss, model_state, losses = train(
        model, lr, epochs, train_dataloader, val_dataloader, show_loss
    )
    
    run_path = f"{weight_path}/run_{run_number}"
    if not os.path.isdir(run_path):
        os.makedirs(run_path)
    
    if val_dataloader is None:
        torch.save(
            model_state, 
            f"{run_path}/final-{best_epoch+1}-{val_loss}.pt"
        )
    else:
        torch.save(
            model_state, 
            f"{run_path}/{best_epoch+1}-{val_loss}.pt"
        )
        
    return np.asarray(losses)

In [7]:
def predict(weight_path, k_gmf, k_mlp, layer_sizes, alpha, out_path, out_name):
    trained_model = NeuFM(
        GMF(users, movies, k_gmf),
        MLP(users, movies, k_mlp, layer_sizes),
        alpha,
    ).cuda()
    
    trained_model.load_state_dict(torch.load(weight_path))
    trained_model.cuda().eval()
    
    test_dataloader = DataLoader(
        RatingsDataset(
            "train_data/test.csv",
            "user_id",
            "movie_id",
        ), 
        batch_size=1024,
        num_workers=12
    )
    
    test_predictions = []

    for vus, vis in test_dataloader:
        vus = vus.cuda()
        vis = vis.cuda()

        pred = torch.clip(trained_model(vus, vis), 1, 5).cpu().ravel().tolist()
        test_predictions += pred
        
    test_csv = pd.read_csv("../../data/test_data.csv")
    
    out_df = pd.DataFrame.from_dict(
        {
            "id": list(test_csv["id"]),
            "rating": test_predictions
        }
    )

    out_df.to_csv(f"{out_path}/{out_name}.csv", index=False)
    out_df.head()

# Train

In [ ]:
losses_fit = []

for run_number, (k_gmf, k_mlp, layers, lr, epochs) in tqdm(enumerate([
    (8, 16, [16, 8], 0.0001, 75),
    (8, 16, [16, 8], 0.0006, 50),
    (8, 16, [16, 8], 0.0008, 50),
    (16, 16, [16, 8], 0.001, 50),
    (16, 32, [32, 16, 8], 0.001, 50),
    (32, 16, [16, 8], 0.001, 50),
    (32, 16, [16, 8], 0.005, 50), #*
    (32, 16, [16, 8], 0.01, 50),
    (32, 32, [32, 16, 8], 0.01, 50),
    (32, 32, [32, 16, 8], 0.01, 100),
])):
    print(f"{k_gmf=} {k_mlp=} {layers=} {lr=} {epochs=}")
    losses_fit.append(
        fit(
            train_dataloader, val_dataloader,
            k_gmf,
            k_mlp,
            layers,
            0.5,
            lr,
            epochs,
            weight_path="/home/nubol23/Documents/NCF_weights_multiple",
            run_number=run_number,
            random_state=2,
            show_loss=False,
        )
    )

0it [00:00, ?it/s]

k_gmf=8 k_mlp=16 layers=[16, 8] lr=0.0001 epochs=75


  0%|          | 0/75 [00:00<?, ?it/s]

k_gmf=8 k_mlp=16 layers=[16, 8] lr=0.0006 epochs=50


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

k_gmf=32 k_mlp=32 layers=[32, 16, 8] lr=0.01 epochs=50


  0%|          | 0/50 [00:00<?, ?it/s]

k_gmf=32 k_mlp=32 layers=[32, 16, 8] lr=0.01 epochs=100


  0%|          | 0/100 [00:00<?, ?it/s]

## Total fit

In [ ]:
total_dataloader = DataLoader(
    RatingsDataset(
        "train_data/total.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=12,
    shuffle=True,
)

In [ ]:
# k_gmf, k_mlp, layers, lr, epochs = (16, 16, [16, 8], 0.001, 16)
k_gmf, k_mlp, layers, lr, epochs = (32, 16, [16, 8], 0.001, 12)

fit(
    total_dataloader, val_dataloader=None,
    k_gmf=k_gmf,
    k_mlp=k_mlp,
    layer_sizes=layers,
    alpha=0.5,
    lr=lr,
    epochs=epochs,
    weight_path="/home/nubol23/Documents/NCF_dot_weights_multiple/",
    run_number=5,
#     random_state=2,
)

## Predict

In [ ]:
out_path = "/home/nubol23/Desktop/Codes/USP/SCC5966/kaggle/notebooks/Project-NeuMF/outputs_csv/neumf_dot"
out_name = "neumf_dot_8"

predict(
    "/home/nubol23/Documents/NCF_dot_weights_multiple/run_5/final-12-0.7520240861793046.pt",
    k_gmf=k_gmf,
    k_mlp=k_mlp,
    layer_sizes=layers,
    alpha=0.5,
    out_path=out_path,
    out_name=out_name,
)

In [ ]:
pd.read_csv(f"{out_path}/{out_name}.csv")